# 3D U-net


Based on this <a href="https://www.mdpi.com/2076-3417/9/3/404/htm#sec2-applsci-09-00404"> article</a> with implementation following this <a href="https://github.com/mrkolarik/3D-brain-segmentation/blob/master/3D-unet.py"> git</a> repository.

In [1]:
#Setting up the model

import os
import numpy as np
import keras
import tensorflow as tf

np.random.seed(256)
tf.set_random_seed(256)

from keras.models import Model
from keras.layers import (Input, concatenate, Conv3D, MaxPooling3D,
            Conv3DTranspose, AveragePooling3D, ZeroPadding3D)
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras import backend as K
from keras.regularizers import l2
from keras.utils import plot_model

K.set_image_data_format('channels_last')

shape = (16,256,256,1) #because we have 1 channel
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def get_unet():
    inputs = Input(shape)
    conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)

    conv4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(conv4)

    conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv3DTranspose(256, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv5), conv4], axis=4)
    conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv3DTranspose(128, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv6), conv3], axis=4)
    conv7 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv3DTranspose(64, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv7), conv2], axis=4)
    conv8 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv8), conv1], axis=4)
    conv9 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv3D(1, (1, 1, 1), activation='sigmoid')(conv9)


    model = Model(inputs=[inputs], outputs=[conv10])

    #model.summary()

    model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

    return model

Unet = get_unet()
Unet.save('Unet.h5')

print("We made a model")

Using TensorFlow backend.
W0722 19:37:39.122041   212 deprecation_wrapper.py:119] From c:\users\carine\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 19:37:39.136035   212 deprecation.py:323] From c:\users\carine\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\nn_impl.py:181: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0722 19:37:39.590243   212 deprecation_wrapper.py:119] From c:\users\carine\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0722 19:37:39.592238   212 deprecation_wrapper.py:119] From c:\users\ca

We made a model


In [2]:
#Loading the data

import os

def get_data(Test_or_Train, img_type, n):
    imgs = []
    name = {'T1' : 'sub-{}_T1w.npy', 
            'grey' : 'c1sub-{}_T1w.npy', 
            'white' : 'c2sub-{}_T1w.npy',
            'csf' : 'c3sub-{}_T1w.npy'}
        
    for i in range(n):
        i += 1
        
        if Test_or_Train == 'Test':
            i += 60
        
        if i < 10:
            numb = '0' + str(i)
        else:
            numb = str(i)
        
        if img_type == 'T1':
            subfolder = 'imgs'
            filename = os.path.join(Test_or_Train,subfolder,name[img_type].format(numb))
        else: 
            subfolder = 'masks'
            filename = os.path.join(Test_or_Train,subfolder,img_type,name[img_type].format(numb))
            
        print('Loading {}'.format(name[img_type].format(numb)))
        
        img = np.load(filename)
        
        #Normalize the images to values in [0,1]
        imgNorm = img / img.max()
        imgNorm = np.expand_dims(imgNorm, axis = 3) #To add the channel dimension (of length 1)
        imgs.append(imgNorm)
        
    print('\n Loaded {} {} images of type {}'.format(n, Test_or_Train, img_type))
    print('-'*30)
    return imgs

x_train = get_data('Train', 'T1', 60)
y_train = get_data('Train', 'grey', 60)

Loading sub-01_T1w.npy
Loading sub-02_T1w.npy
Loading sub-03_T1w.npy
Loading sub-04_T1w.npy
Loading sub-05_T1w.npy
Loading sub-06_T1w.npy
Loading sub-07_T1w.npy
Loading sub-08_T1w.npy
Loading sub-09_T1w.npy
Loading sub-10_T1w.npy
Loading sub-11_T1w.npy
Loading sub-12_T1w.npy
Loading sub-13_T1w.npy
Loading sub-14_T1w.npy
Loading sub-15_T1w.npy
Loading sub-16_T1w.npy
Loading sub-17_T1w.npy
Loading sub-18_T1w.npy
Loading sub-19_T1w.npy
Loading sub-20_T1w.npy
Loading sub-21_T1w.npy
Loading sub-22_T1w.npy
Loading sub-23_T1w.npy
Loading sub-24_T1w.npy
Loading sub-25_T1w.npy
Loading sub-26_T1w.npy
Loading sub-27_T1w.npy
Loading sub-28_T1w.npy
Loading sub-29_T1w.npy
Loading sub-30_T1w.npy
Loading sub-31_T1w.npy
Loading sub-32_T1w.npy
Loading sub-33_T1w.npy
Loading sub-34_T1w.npy
Loading sub-35_T1w.npy
Loading sub-36_T1w.npy
Loading sub-37_T1w.npy
Loading sub-38_T1w.npy
Loading sub-39_T1w.npy
Loading sub-40_T1w.npy
Loading sub-41_T1w.npy
Loading sub-42_T1w.npy
Loading sub-43_T1w.npy
Loading sub

c:\users\carine\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide


Loading c1sub-23_T1w.npy
Loading c1sub-24_T1w.npy
Loading c1sub-25_T1w.npy
Loading c1sub-26_T1w.npy
Loading c1sub-27_T1w.npy
Loading c1sub-28_T1w.npy
Loading c1sub-29_T1w.npy
Loading c1sub-30_T1w.npy
Loading c1sub-31_T1w.npy
Loading c1sub-32_T1w.npy
Loading c1sub-33_T1w.npy
Loading c1sub-34_T1w.npy
Loading c1sub-35_T1w.npy
Loading c1sub-36_T1w.npy
Loading c1sub-37_T1w.npy
Loading c1sub-38_T1w.npy
Loading c1sub-39_T1w.npy
Loading c1sub-40_T1w.npy
Loading c1sub-41_T1w.npy
Loading c1sub-42_T1w.npy
Loading c1sub-43_T1w.npy
Loading c1sub-44_T1w.npy
Loading c1sub-45_T1w.npy
Loading c1sub-46_T1w.npy
Loading c1sub-47_T1w.npy
Loading c1sub-48_T1w.npy
Loading c1sub-49_T1w.npy
Loading c1sub-50_T1w.npy
Loading c1sub-51_T1w.npy
Loading c1sub-52_T1w.npy
Loading c1sub-53_T1w.npy
Loading c1sub-54_T1w.npy
Loading c1sub-55_T1w.npy
Loading c1sub-56_T1w.npy
Loading c1sub-57_T1w.npy
Loading c1sub-58_T1w.npy
Loading c1sub-59_T1w.npy
Loading c1sub-60_T1w.npy

 Loaded 60 Train images of type grey
------------

In [ ]:
#Unet.summary()

print('-'*30)
print('Fitting model...')
print('-'*30)

Unet.fit([x_train], [y_train], batch_size=1, epochs=2, verbose=1, shuffle=True) #validation_split=0.10
Unet.save('Unet_trained.h5')

print('-'*30)
print('Training finished')
print('-'*30)

------------------------------
Fitting model...
------------------------------


W0722 16:21:38.921216  4196 deprecation_wrapper.py:119] From c:\users\carine\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0722 16:21:39.572629  4196 deprecation_wrapper.py:119] From c:\users\carine\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0722 16:21:40.282326  4196 deprecation_wrapper.py:119] From c:\users\carine\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0722 16:21:40.388264  4196 deprecation_wrapper.py:119] From c:\users\carine\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.C

Epoch 1/2
